In [12]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [14]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')
#data = pd.read_csv("c:/Users/Mercy/OneDrive/Documents/ServiceNow classes/OneDrive/AI Training Docs/Labs/week 4/lab-natural-language-processing/data/kg_train.csv", encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [15]:
# Your code
# divide the training and test set into 2 partitions
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])

## Data Preprocessing

In [16]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [17]:
# Your code
# remove inline javascript/css
import re

def remove_inline_js_css(text):
    # Remove <script>...</script> and <style>...</style> blocks
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL | re.IGNORECASE)
    return text

# Apply to your DataFrame column (replace 'text' with your actual column name)
data['clean_text'] = data['text'].apply(remove_inline_js_css)
print(data['clean_text'])

0      DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                               Will do.
2      Nora--Cheryl has emailed dozens of memos about...
3      Dear Sir=2FMadam=2C I know that this proposal ...
4                                                    fyi
                             ...                        
995    So what's the latest? It sounds contradictory ...
996    TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...
997    Barb I will call to explain. Are you back in t...
998      Yang on travelNot free tonite.May work tomorrow
999    sbwhoeopSunday February 21 2010 7:42 PMHShaunH...
Name: clean_text, Length: 1000, dtype: object


In [18]:
# remove html comments
import re

def remove_html_comments(text):
    return re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)

# Apply to your DataFrame column (replace 'clean_text' with your actual column name)
data['clean_text'] = data['clean_text'].apply(remove_html_comments)
print(data['clean_text'])

0      DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                               Will do.
2      Nora--Cheryl has emailed dozens of memos about...
3      Dear Sir=2FMadam=2C I know that this proposal ...
4                                                    fyi
                             ...                        
995    So what's the latest? It sounds contradictory ...
996    TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...
997    Barb I will call to explain. Are you back in t...
998      Yang on travelNot free tonite.May work tomorrow
999    sbwhoeopSunday February 21 2010 7:42 PMHShaunH...
Name: clean_text, Length: 1000, dtype: object


In [19]:
def remove_html_tags(text):
    return re.sub(r'<.*?>', '', text)

data['clean_text'] = data['clean_text'].apply(remove_html_tags)
print(data['clean_text'])

0      DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                               Will do.
2      Nora--Cheryl has emailed dozens of memos about...
3      Dear Sir=2FMadam=2C I know that this proposal ...
4                                                    fyi
                             ...                        
995    So what's the latest? It sounds contradictory ...
996    TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...
997    Barb I will call to explain. Are you back in t...
998      Yang on travelNot free tonite.May work tomorrow
999    sbwhoeopSunday February 21 2010 7:42 PMHShaunH...
Name: clean_text, Length: 1000, dtype: object


- Remove all the special characters

In [20]:
# Your code
# remove all the special characters
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

data['clean_text'] = data['clean_text'].apply(remove_special_characters)
print(data['clean_text'])
    

0      DEAR SIR STRICTLY A PRIVATE BUSINESS PROPOSAL ...
1                                                Will do
2      NoraCheryl has emailed dozens of memos about H...
3      Dear Sir2FMadam2C I know that this proposal mi...
4                                                    fyi
                             ...                        
995    So whats the latest It sounds contradictory an...
996    TRANSFER OF 3675900000 MILLION POUNDS TO YOURA...
997    Barb I will call to explain Are you back in th...
998       Yang on travelNot free toniteMay work tomorrow
999    sbwhoeopSunday February 21 2010 742 PMHShaunH ...
Name: clean_text, Length: 1000, dtype: object


- Remove numbers

In [22]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

data['clean_text'] = data['clean_text'].apply(remove_numbers)
print(data['clean_text'])

0      DEAR SIR STRICTLY A PRIVATE BUSINESS PROPOSAL ...
1                                                Will do
2      NoraCheryl has emailed dozens of memos about H...
3      Dear SirFMadamC I know that this proposal migh...
4                                                    fyi
                             ...                        
995    So whats the latest It sounds contradictory an...
996    TRANSFER OF  MILLION POUNDS TO YOURACCOUNTMy n...
997    Barb I will call to explain Are you back in th...
998       Yang on travelNot free toniteMay work tomorrow
999    sbwhoeopSunday February    PMHShaunH Just talk...
Name: clean_text, Length: 1000, dtype: object


- Remove all single characters

In [27]:
def remove_single_characters(text):
    # Remove single characters surrounded by spaces
    return re.sub(r'\b\w\b', '', text)

data['clean_text'] = data['clean_text'].apply(remove_single_characters)
print(data['clean_text'])

0      DEAR SIR STRICTLY  PRIVATE BUSINESS PROPOSAL  ...
1                                                Will do
2      NoraCheryl has emailed dozens of memos about H...
3      Dear SirFMadamC  know that this proposal might...
4                                                    fyi
                             ...                        
995    So whats the latest It sounds contradictory an...
996    TRANSFER OF  MILLION POUNDS TO YOURACCOUNTMy n...
997    Barb  will call to explain Are you back in the...
998       Yang on travelNot free toniteMay work tomorrow
999    sbwhoeopSunday February    PMHShaunH Just talk...
Name: clean_text, Length: 1000, dtype: object


- Remove single characters from the start

In [29]:
def remove_single_char_start(text):
    return re.sub(r'^\w\s+', '', text)

# Apply to your DataFrame column
data['clean_text'] = data['clean_text'].apply(remove_single_char_start)
print(data['clean_text'])

0      DEAR SIR STRICTLY  PRIVATE BUSINESS PROPOSAL  ...
1                                                Will do
2      NoraCheryl has emailed dozens of memos about H...
3      Dear SirFMadamC  know that this proposal might...
4                                                    fyi
                             ...                        
995    So whats the latest It sounds contradictory an...
996    TRANSFER OF  MILLION POUNDS TO YOURACCOUNTMy n...
997    Barb  will call to explain Are you back in the...
998       Yang on travelNot free toniteMay work tomorrow
999    sbwhoeopSunday February    PMHShaunH Just talk...
Name: clean_text, Length: 1000, dtype: object


- Substitute multiple spaces with single space

In [30]:
def substitute_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

# Apply to your DataFrame column
data['clean_text'] = data['clean_text'].apply(substitute_multiple_spaces)
print(data['clean_text'])

0      DEAR SIR STRICTLY PRIVATE BUSINESS PROPOSAL AM...
1                                                Will do
2      NoraCheryl has emailed dozens of memos about H...
3      Dear SirFMadamC know that this proposal might ...
4                                                    fyi
                             ...                        
995    So whats the latest It sounds contradictory an...
996    TRANSFER OF MILLION POUNDS TO YOURACCOUNTMy na...
997    Barb will call to explain Are you back in the ...
998       Yang on travelNot free toniteMay work tomorrow
999    sbwhoeopSunday February PMHShaunH Just talked ...
Name: clean_text, Length: 1000, dtype: object


- Remove prefixed 'b'

In [31]:
def remove_prefixed_b(text):
    return re.sub(r"^b['\"]?", '', text)

# Apply to your DataFrame column
data['clean_text'] = data['clean_text'].apply(remove_prefixed_b)
print(data['clean_text'])

0      DEAR SIR STRICTLY PRIVATE BUSINESS PROPOSAL AM...
1                                                Will do
2      NoraCheryl has emailed dozens of memos about H...
3      Dear SirFMadamC know that this proposal might ...
4                                                    fyi
                             ...                        
995    So whats the latest It sounds contradictory an...
996    TRANSFER OF MILLION POUNDS TO YOURACCOUNTMy na...
997    Barb will call to explain Are you back in the ...
998       Yang on travelNot free toniteMay work tomorrow
999    sbwhoeopSunday February PMHShaunH Just talked ...
Name: clean_text, Length: 1000, dtype: object


- Convert to Lowercase
- Observation: Line 996 changed to lower case

In [ ]:
data['clean_text'] = data['clean_text'].str.lower()
print(data['clean_text'])

0      dear sir strictly private business proposal am...
1                                                will do
2      noracheryl has emailed dozens of memos about h...
3      dear sirfmadamc know that this proposal might ...
4                                                    fyi
                             ...                        
995    so whats the latest it sounds contradictory an...
996    transfer of million pounds to youraccountmy na...
997    barb will call to explain are you back in the ...
998       yang on travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh just talked ...
Name: clean_text, Length: 1000, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [33]:
# Your code
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return ' '.join(filtered)

data['clean_text'] = data['clean_text'].apply(remove_stopwords)
print(data['clean_text'])

0      dear sir strictly private business proposal mi...
1                                                       
2      noracheryl emailed dozens memos haiti weekend ...
3      dear sirfmadamc know proposal might surprise e...
4                                                    fyi
                             ...                        
995    whats latest sounds contradictory af decide sh...
996    transfer million pounds youraccountmy name mre...
997                       barb call explain back country
998          yang travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh talked shaun...
Name: clean_text, Length: 1000, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [40]:
# Your code
from nltk.stem import WordNetLemmatizer

In [42]:
lemmatizer = WordNetLemmatizer()
# Create a new column with lemmatized tokens (list of words)
data['lemmatized_tokens'] = data['clean_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x.split()])
print("Lemmatized Tokens:", data['lemmatized_tokens'])

Lemmatized Tokens: 0      [dear, sir, strictly, private, business, propo...
1                                                     []
2      [noracheryl, emailed, dozen, memo, haiti, week...
3      [dear, sirfmadamc, know, proposal, might, surp...
4                                                  [fyi]
                             ...                        
995    [whats, latest, sound, contradictory, af, deci...
996    [transfer, million, pound, youraccountmy, name...
997                 [barb, call, explain, back, country]
998    [yang, travelnot, free, tonitemay, work, tomor...
999    [sbwhoeopsunday, february, pmhshaunh, talked, ...
Name: lemmatized_tokens, Length: 1000, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [43]:
from collections import Counter

# For ham messages (label == 0)
ham_words = [word for tokens in data[data['label'] == 0]['lemmatized_tokens'] for word in tokens]
ham_top10 = Counter(ham_words).most_common(10)
print("Top 10 words in ham messages:", ham_top10)

# For spam messages (label == 1)
spam_words = [word for tokens in data[data['label'] == 1]['lemmatized_tokens'] for word in tokens]
spam_top10 = Counter(spam_words).most_common(10)
print("Top 10 words in spam messages:", spam_top10)

Top 10 words in ham messages: [('u', 116), ('pm', 115), ('would', 106), ('state', 103), ('president', 94), ('call', 91), ('time', 84), ('percent', 77), ('secretary', 76), ('work', 73)]
Top 10 words in spam messages: [('money', 920), ('account', 794), ('bank', 745), ('fund', 703), ('u', 550), ('business', 473), ('transaction', 416), ('country', 406), ('transfer', 392), ('million', 385)]


## Extra features

In [45]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

# Make sure to use the correct column for preprocessed text. If you want to use 'clean_text', change below accordingly.
train['money_mark'] = train['text'].str.contains(money_simbol_list, case=False, na=False)*1
train['suspicious_words'] = train['text'].str.contains(suspicious_words, case=False, na=False)*1
train['text_len'] = train['text'].apply(lambda x: len(x)) 

test['money_mark'] = test['text'].str.contains(money_simbol_list, case=False, na=False)*1
test['suspicious_words'] = test['text'].str.contains(suspicious_words, case=False, na=False)*1
test['text_len'] = test['text'].apply(lambda x: len(x)) 

train.head()

,text,label,money_mark,suspicious_words,text_len
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,1,1,1609
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,1,1,3123
971,Will do.,0,1,0,8
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,1,1,530
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,1,1,2126


## How would work the Bag of Words with Count Vectorizer concept?

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'clean_text' column to create the bag-of-words representation
X_bow = vectorizer.fit_transform(data['clean_text'])

# Show the shape of the resulting matrix (documents x vocabulary size)
print("Bag of Words matrix shape:", X_bow.shape)

# Show feature names (words)
print("Sample feature names:", vectorizer.get_feature_names_out()[:10])

Bag of Words matrix shape: (1000, 19354)
Sample feature names: ['aac' 'aaclocated' 'aae' 'aag' 'aaronovitchon' 'abacha' 'abachabefore'
 'abachac' 'abachace' 'abachaco']


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [47]:
# Load the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Vectorize the entire 'clean_text' column
X_tfidf = tfidf_vectorizer.fit_transform(data['clean_text'])

# Print the shape of the vectorized dataset
print("TF-IDF matrix shape:", X_tfidf.shape)

TF-IDF matrix shape: (1000, 19354)


## And the Train a Classifier?

In [48]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Prepare features and labels
X = X_tfidf
y = data['label']

# Split into train and test sets (using the same indices as before)
X_train = X[train.index]
X_test = X[test.index]
y_train = train['label']
y_test = test['label']

# Train the classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.955
              precision    recall  f1-score   support

           0       1.00      0.92      0.96       112
           1       0.91      1.00      0.95        88

    accuracy                           0.95       200
   macro avg       0.95      0.96      0.95       200
weighted avg       0.96      0.95      0.96       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Example: Try TF-IDF with unigrams and bigrams + extra features (already shown in cell 45)
# You can experiment with different feature combinations below:

# For spam messages (label == 1)
spam_words = [word for tokens in data[data['label'] == 1]['lemmatized_tokens'] for word in tokens]
spam_top10 = Counter(spam_words).most_common(10)
print("Top 10 words in spam messages:", spam_top10)

# Try character n-grams + word n-grams + extra features
vectorizer_word = TfidfVectorizer(ngram_range=(1,2), analyzer='word', min_df=2)
vectorizer_char = TfidfVectorizer(ngram_range=(3,5), analyzer='char', min_df=2)

X_word = vectorizer_word.fit_transform(data['clean_text'])
X_char = vectorizer_char.fit_transform(data['clean_text'])

# Combine all features using train and test DataFrames for extra features
extra_features_train = train[['money_mark', 'suspicious_words', 'text_len']].values
extra_features_test = test[['money_mark', 'suspicious_words', 'text_len']].values

X_train_all = hstack([X_word[train.index], X_char[train.index], csr_matrix(extra_features_train)])
X_test_all = hstack([X_word[test.index], X_char[test.index], csr_matrix(extra_features_test)])
y_train = train['label']
y_test = test['label']

# Train and evaluate
clf = MultinomialNB()
clf.fit(X_train_all, y_train)
y_pred = clf.predict(X_test_all)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Top 10 words in spam messages: [('money', 920), ('account', 794), ('bank', 745), ('fund', 703), ('u', 550), ('business', 473), ('transaction', 416), ('country', 406), ('transfer', 392), ('million', 385)]
Accuracy: 0.575
              precision    recall  f1-score   support

           0       1.00      0.24      0.39       112
           1       0.51      1.00      0.67        88

    accuracy                           0.57       200
   macro avg       0.75      0.62      0.53       200
weighted avg       0.78      0.57      0.51       200

